In [ ]:
from twitter_api import twitter_reader as tr
from data_storage.db_manager import save_tweets, DBConfig, read_tweets, clear_tweet_table

In [2]:
tr_conf = tr.Config()
response = tr.query('ferrari', tr_conf)

got 10 tweets.


In [3]:
db_conf = DBConfig()

In [4]:
clear_tweet_table(db_conf)
save_tweets(response['data'], db_conf)

In [5]:
tweets = read_tweets(db_conf)

In [6]:
tweets

[{'id': '1379731923145256962',
  'author_id': '929983174565908480',
  'created_at': '2021-04-07T09:44:59.000Z',
  'text': '@ausgrandprix Has to be Ferrari'},
 {'id': '1379732328814100481',
  'author_id': '831763872747048960',
  'created_at': '2021-04-07T09:46:36.000Z',
  'text': 'RT @sarcastic_bobby: @oj_deji That babe na my ex...A ti Ferrari https://t.co/A7ytWEIjZn'},
 {'id': '1379732282685206528',
  'author_id': '2285395853',
  'created_at': '2021-04-07T09:46:25.000Z',
  'text': 'RT @_DAGOSPIA_: LO SCIAMANO DI PIAZZA MONTECITORIO, HERMES FERRARI, HA UN PASSATO DI RISSE E ARRESTI https://t.co/CU3xovAi6c https://t.co/i…'},
 {'id': '1379732193283600386',
  'author_id': '40190755',
  'created_at': '2021-04-07T09:46:03.000Z',
  'text': 'RT @qendra_e: Në një foto e shkrepur një ditë të marte shumë vite më parë në Francineto dhe të publikuar nga Lucia Martino shfaqet Ernest K…'},
 {'id': '1379732156533063680',
  'author_id': '2243508517',
  'created_at': '2021-04-07T09:45:54.000Z',
  'text'